The code below helps calculate the profit for Betting Boosts

In [ ]:
underdog_odds = 300
favorite_odds = 800

betting_boost = 0.3

# rh_ud = 32/100
rh_fav = 79/100
underdog_stake = 25    # how much you want to put on the underdog



underdog = 100/(100 + underdog_odds)
favorite = favorite_odds/(100 + favorite_odds)


print("Total Odds")
print(underdog)
print(favorite)
print(underdog + favorite)
underdog_odds = underdog_odds * (1 + betting_boost)
favorite_odds = favorite_odds / (1 + betting_boost)
underdog = 100/(100 + underdog_odds)
favorite = favorite_odds/(100 + favorite_odds)

print("\nAdjusted Odds with Boost")
print(underdog)
print(favorite)
print(underdog + favorite)

def hedge_stake(underdog_stake):
    # Convert to implied probabilities
    p_u = underdog
    p_f = rh_fav
    # How much to bet on favorite
    s_f = underdog_stake * (p_f / p_u)
    # Guaranteed profit (pre-fee), only valid if p_u + p_f < 1
    profit = underdog_stake * (1 - p_u - p_f) / p_u
    return p_u, p_f, s_f, profit

p_u, p_f, s_f, profit = hedge_stake(underdog_stake)

print(f"\nUnderdog implied probability: {p_u:.3f}")
print(f"Favorite implied probability: {p_f:.3f}")
print(f"Bet ${s_f:.2f} on the favorite")
print(f"Guaranteed profit (pre-fee): ${profit:.2f}")
print(f"Sum: {p_u + p_f:.3f}")


Total Odds
0.23809523809523808
0.8888888888888888
1.126984126984127

Adjusted Odds with Boost
0.1937984496124031
0.8602150537634409
1.054013503375844

Underdog implied probability: 0.194
Favorite implied probability: 0.790
Bet $101.91 on the favorite
Guaranteed profit (pre-fee): $2.09
Sum: 0.984


: 

The code block below calculates the maximum profit from free bets

In [9]:
# ---- Inputs (your style) ----
underdog_odds = 140          # American odds for the underdog (e.g., +120)
favorite_odds = 175          # American odds for the favorite (use sign correctly: + or -)
betting_boost = 0          # 50% profit boost on the underdog (set 0 if none)
rh_fav = 64/100              # If hedging on an exchange (e.g., 0.57 price). Set None to use favorite_odds.
underdog_stake = 5          # Amount you place on the underdog side
free_bet = True              # <<< set True if the underdog stake is a FREE BET (stake not returned)

# ---- Helpers ----
def boosted_american(odds, boost):
    # Apply profit boost to American odds
    if odds >= 0:
        return odds * (1 + boost)      # +A -> (1+b)*A
    else:
        return odds / (1 + boost)      # -A -> A/(1+b)

def american_to_prob(odds):
    # Convert American odds to implied probability
    if odds >= 0:
        return 100 / (odds + 100)
    else:
        A = abs(odds)
        return A / (A + 100)

# ---- Compute implied probabilities (after boost on UD only) ----
boosted_ud_odds = boosted_american(underdog_odds, betting_boost)
p_u = american_to_prob(boosted_ud_odds)            # underdog (boosted) implied probability

if rh_fav is not None:
    p_f = float(rh_fav)                            # already a probability (e.g., 0.57)
else:
    p_f = american_to_prob(favorite_odds)          # from American odds

print("Total Odds (pre calc)")
print(p_u)
print(p_f)
print(p_u + p_f)

# ---- Hedge sizing & profit ----
if not free_bet:
    # CASH BET on the underdog (stake returned if win): your original logic
    # Hedge so payouts match: s_f = s_ud * (p_f / p_u)
    s_f = underdog_stake * (p_f / p_u)
    # Guaranteed profit (pre-fee), valid if p_u + p_f < 1
    profit = underdog_stake * (1 - p_u - p_f) / p_u

    print("\nAdjusted Odds with Boost (cash bet on UD)")
    print(p_u)
    print(p_f)
    print(p_u + p_f)
    print(f"\nStake on underdog: ${underdog_stake:.2f}")
    print(f"Stake on favorite (hedge): ${s_f:.2f}")
    print(f"Guaranteed profit (pre-fee): ${profit:.2f}")

else:
    # FREE BET on the underdog (stake NOT returned if win)
    # Equal-profit hedge: s_f = F * ( (1/p_u - 1) ) / (1/p_f) = F * ((1 - p_u)/p_u) * p_f
    s_f = underdog_stake * ((1 - p_u) / p_u) * p_f

    # Profit is the same in both outcomes:
    # If UD wins:  (1/p_u - 1)*F - s_f
    # If Fav wins: s_f*(1/p_f - 1)
    profit_ud = (1/p_u - 1) * underdog_stake - s_f
    profit_fv = s_f * (1/p_f - 1)
    # Closed-form (sanity): profit = F * ((1 - p_u)/p_u) * (1 - p_f)
    profit_closed = underdog_stake * ((1 - p_u)/p_u) * (1 - p_f)

    print("\nAdjusted Odds with Boost (FREE BET on UD)")
    print(p_u)
    print(p_f)
    print(p_u + p_f)
    print(f"\nStake (FREE BET) on underdog: ${underdog_stake:.2f}")
    print(f"Stake on favorite (hedge): ${s_f:.2f}")
    print(f"Guaranteed profit if UD wins: ${profit_ud:.2f}")
    print(f"Guaranteed profit if Fav wins: ${profit_fv:.2f}")
    print(f"Closed-form profit check:   ${profit_closed:.2f}")


Total Odds (pre calc)
0.4166666666666667
0.64
1.0566666666666666

Adjusted Odds with Boost (FREE BET on UD)
0.4166666666666667
0.64
1.0566666666666666

Stake (FREE BET) on underdog: $5.00
Stake on favorite (hedge): $4.48
Guaranteed profit if UD wins: $2.52
Guaranteed profit if Fav wins: $2.52
Closed-form profit check:   $2.52
